In [2]:
import polars as pl

In [3]:
wos = pl.read_excel("samples/tiger_WoS_28.01.25_kathi.xls")
scopus = pl.read_csv("samples/Tiger_scopus_Kathi_13.1.25.csv")

Could not determine dtype for column 4, falling back to string
Could not determine dtype for column 8, falling back to string
Could not determine dtype for column 10, falling back to string
Could not determine dtype for column 12, falling back to string
Could not determine dtype for column 13, falling back to string
Could not determine dtype for column 14, falling back to string
Could not determine dtype for column 18, falling back to string
Could not determine dtype for column 20, falling back to string
Could not determine dtype for column 22, falling back to string
Could not determine dtype for column 26, falling back to string
Could not determine dtype for column 31, falling back to string
Could not determine dtype for column 34, falling back to string
Could not determine dtype for column 35, falling back to string
Could not determine dtype for column 36, falling back to string
Could not determine dtype for column 42, falling back to string
Could not determine dtype for column 43, f

In [25]:
def join_reduce(files) -> pl.DataFrame:
    first = load_search(files[0])
    for f in map(load_search, files[1:]):
        first = join(first, f)
    return first


def join(left: pl.DataFrame, right=None) -> pl.DataFrame:
    if right is None:
        return left

    joined = left.join(right, on=pl.col("Title").str.to_lowercase().str.strip_chars('"'), how="full")
    return joined.select(
        [
            pl.coalesce(["Authors", "Authors_right"]),
            pl.coalesce(["Title", "Title_right"]),
            pl.coalesce(["DOI", "DOI_right"]),
            pl.coalesce(["Year", "Year_right"]),
            pl.concat_str(
                ["File", "File_right"],
                ignore_nulls=True,
                separator=",",
            ),
        ]
    )


def load_search(filename: str) -> pl.DataFrame:
    if filename.endswith(".csv"):
        return load_scopus(filename)
    else:
        return load_wos(filename)


def load_scopus(filename: str) -> pl.DataFrame:
    df = pl.read_csv(filename)
    return df.select(["Authors", "Title", "DOI", "Year"]).with_columns(
        pl.lit("scopus").alias("Search Engine"), pl.lit(filename).alias("File")
    )


def load_wos(filename: str) -> pl.DataFrame:
    df = pl.read_excel(filename, columns=["Authors", "Article Title", "DOI", "Publication Year"])
    df = df.select(
        [
            "Authors",
            pl.col("Article Title").alias("Title"),
            "DOI",
            pl.col("Publication Year").alias("Year"),
        ]
    )
    df = df.with_columns(lowered=pl.col("Title").str.to_lowercase()).unique("lowered").drop("lowered")
    return df.with_columns(pl.lit("wos").alias("Search Engine"), pl.lit(filename).alias("File"))


In [26]:
a = load_search("samples/wos_1.xls")
b = load_search("samples/wos_1.xls")

j = join(a, b)

In [27]:
dups = j.filter(j.is_duplicated())